In [23]:
import numpy as np
import os
import time
import pandas as pd
import re
import json
import keras
import glob

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import classification_report, confusion_matrix

In [24]:
# Build the corpus and sequences
labelToName = { 0 : 'PREMISE', 1 : 'CONCLUSION' }
namesInLabelOrder = ['PREMISE', 'CONCLUSION']
os.chdir('E:\git\SmartLawML\work-area')
df=pd.read_csv('preprocessed.csv')
X=df.text.tolist()
labels=df.loc[:,['label']]


def preprocess(temp):
    temp=re.sub("\s\s+"," ",temp)    #Replacing multiple spaces with one
    temp=temp.replace(u'\xa0', ' ').encode('utf-8')  #Replacing non-breaking space    
    temp=temp.decode().split(' ')    #Converting a string into list of words seperated by space charcter
    return temp

#print(X[1])
X = [preprocess(t) for t in X]  
#print("\n\n\n After Preprocess:\n\n",X[1])



In [25]:
X=df.text.tolist()
labels=df.loc[:,['label']]


def preprocess(temp):
    temp=re.sub("\s\s+"," ",temp)    #Replacing multiple spaces with one
    temp=temp.replace(u'\xa0', ' ').encode('utf-8')  #Replacing non-breaking space    
    temp=temp.decode().split(' ')    #Converting a string into list of words seperated by space charcter
    return temp

#print(X[1])
X = [preprocess(t) for t in X]  
print("\n\n\n After Preprocess:\n\n",X[1])




 After Preprocess:

 ["['prosecution',", "'case',", "'p',", "'complainant',", "'came',", "'house',", "'grazing',", "'sheep',", "'applicant',", "'came',", "'complainant']"]


In [26]:
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=1).split(X, labels)
train_indices, test_indices = next(sss)

def seqlengths(l):
    if isinstance(l,list):
        yield len(l)
        for y in l:
            yield from seqlengths(y)

maxsequenceLength=max(seqlengths(X))
print("Max sequence length:",maxsequenceLength)

Max sequence length: 3076


In [27]:
# Encode the documents
kTokenizer = keras.preprocessing.text.Tokenizer() 
kTokenizer.fit_on_texts(X)
Xencoded = np.array([np.array(xi) for xi in kTokenizer.texts_to_sequences(X)])                

print("Shape Before Padding:",np.shape(Xencoded[22]))
Xencoded=keras.preprocessing.sequence.pad_sequences(Xencoded, maxlen=maxsequenceLength, truncating='post')
print("\nShape After Padding:",np.shape(Xencoded[22]))

labels = np.array(labels)

nWords=len(kTokenizer.word_index)
print("\n nWords:",nWords)

Shape Before Padding: (10,)

Shape After Padding: (3076,)

 nWords: 2739


C:\Users\abc\AppData\Local\Temp\ipykernel_8560\199922491.py:4: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  Xencoded = np.array([np.array(xi) for xi in kTokenizer.texts_to_sequences(X)])


In [30]:
vectorizer = TfidfVectorizer(analyzer=lambda x: x, min_df=1).fit(X)
Xencoded=vectorizer.transform(X)

from sklearn.linear_model import LogisticRegression
results = {}
model = LogisticRegression()
train_x = Xencoded[train_indices]
test_x = Xencoded[test_indices]
train_labels = labels[train_indices]
test_labels = labels[test_indices]
model.fit(train_x, train_labels)
predicted_labels = model.predict(test_x)
results['confusion_matrix'] = confusion_matrix(labels[test_indices], predicted_labels).tolist()
results['classification_report'] = classification_report(labels[test_indices], predicted_labels, digits=4, target_names=namesInLabelOrder, output_dict=True)
print ("\n\nConfusion Matrix:\n",confusion_matrix(labels[test_indices], predicted_labels))
print ("\n\nClassification Report:\n\n",classification_report(labels[test_indices], predicted_labels, digits=4, target_names=namesInLabelOrder))


#os.chdir('D:\\PhD\\Dataset\\Results\\Paper2')
#filename = 'TFIDF-LR-'+time.strftime("%d-%m-%Y-%H-%M-%S")+'.json'
#f = open (filename,'w')
#out = json.dumps(results, ensure_ascii=True, indent=4)
#f.write(out)
#f.close()



Confusion Matrix:
 [[ 44  44]
 [  6 522]]


Classification Report:

               precision    recall  f1-score   support

     PREMISE     0.8800    0.5000    0.6377        88
  CONCLUSION     0.9223    0.9886    0.9543       528

    accuracy                         0.9188       616
   macro avg     0.9011    0.7443    0.7960       616
weighted avg     0.9162    0.9188    0.9091       616



E:\git\SmartLawML\smartlawml-microservice\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
